In [116]:
import sys
import os
import json
import importlib

sys.path.append(os.path.abspath('../utils'))

import tool_agent
importlib.reload(tool_agent)
from tool_agent import ToolAgent

import tool_functions
importlib.reload(tool_functions)
from tool_functions import convert_to_tool

import rag
importlib.reload(rag)
from rag import RAG

from rich.markdown import Markdown as rich_Markdown

## Use a dummy function to test

In [117]:
def get_current_weather(location: str, unit: str):
    """
    Get the current weather in a given location
    
    Args:
        location (str): The city and state, e.g. Madrid, Barcelona
        unit (str): The unit. It can take two values; "Felsius", "Fahrenheit"
    """
    if location == "Charlotte":
        return json.dumps({"Retrieved data": f"the temperature is 25 {unit}"})
    else:
        return json.dumps({"Retrieved data": f"the temperature is 50 {unit}"})


In [118]:
print(get_current_weather(location="Charlotte", unit="Fahrenheit"))
weather_tool = convert_to_tool(get_current_weather)

{"Retrieved data": "the temperature is 25 Fahrenheit"}


In [124]:
tool_agent = ToolAgent(tools=[weather_tool])
temp=tool_agent.run("What is the weather today in Charlotte?")
rich_Markdown(temp)

                                                                                                                   
 <tool_call>                                                                                                       
 {"name": "get_current_weather", "arguments": {"location": "Charlotte", "unit": "Fahrenheit"}, "id": 1}            
 </tool_call>                                                                                                      
 ```The temperature in Charlotte is 25 Fahrenheit.                                                                 
                                                                                                                   


In [98]:
tool_agent = ToolAgent(tools=[weather_tool])
temp=tool_agent.run("What is net income of wells fargo this quarter")
print(temp)

[role: "user"
parts {
  text: "\nYou are a function calling and questions answering AI model. You are provided with function signatures within <tools></tools> XML tags.\nYou may call one or more functions to assist with the user query. Don\'t make assumptions about what values to plug into functions. Pay special attention to the properties \'types\'. You should use those types as in a Python dict.\n\nFor each function call return a json object with function name and arguments within <tool_call></tool_call>\nXML tags as follows:\n\n<tool_call>\n{\"name\": <function-name>,\"arguments\": <args-dict>,  \"id\": <monotonically-increasing-id>}\n</tool_call>\n\nHere are the available tools:\n\n<tools>\n{\"name\": \"get_current_weather\", \"description\": \"\\n    Get the current weather in a given location\\n    \\n    Args:\\n        location (str): The city and state, e.g. Madrid, Barcelona\\n        unit (str): The unit. It can take two values; \\\"Felsius\\\", \\\"Fahrenheit\\\"\\n    \", 

In [99]:
tool_agent = ToolAgent(tools=[weather_tool])
temp=tool_agent.run("What is the weather today in Charlotte and San Francisco?")
print(temp)

[role: "user"
parts {
  text: "\nYou are a function calling and questions answering AI model. You are provided with function signatures within <tools></tools> XML tags.\nYou may call one or more functions to assist with the user query. Don\'t make assumptions about what values to plug into functions. Pay special attention to the properties \'types\'. You should use those types as in a Python dict.\n\nFor each function call return a json object with function name and arguments within <tool_call></tool_call>\nXML tags as follows:\n\n<tool_call>\n{\"name\": <function-name>,\"arguments\": <args-dict>,  \"id\": <monotonically-increasing-id>}\n</tool_call>\n\nHere are the available tools:\n\n<tools>\n{\"name\": \"get_current_weather\", \"description\": \"\\n    Get the current weather in a given location\\n    \\n    Args:\\n        location (str): The city and state, e.g. Madrid, Barcelona\\n        unit (str): The unit. It can take two values; \\\"Felsius\\\", \\\"Fahrenheit\\\"\\n    \", 

```tool_code
<tool_call>
{"name": "get_current_weather", "arguments": {"location": "Charlotte", "unit": "Fahrenheit"}, "id": 1}
</tool_call>
<tool_call>
{"name": "get_current_weather", "arguments": {"location": "San Francisco", "unit": "Fahrenheit"}, "id": 2}
</tool_call>
```Based on the retrieved data, the temperature in Charlotte is 25 Fahrenheit and the temperature in San Francisco is 50 Fahrenheit.


## Use a RAG serach function to test

In [11]:
# read the data, create an RAG class on that data, create the search function
data_path='../data/naf23.pdf'

In [15]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# create a faiss_vector_store
pdf_reader = PdfReader(data_path)
text = ""
chunks=[]

# read the pages
for page in pdf_reader.pages:
    text += page.extract_text()

# text_splitter.create_documents() splits the text into chunks, then change each chunk into a 'Document' object of Langchain
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
chunks=text_splitter.create_documents([text])

# embedding model
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

# faiss_vector_store
faiss_vector_store = FAISS.from_documents(chunks, embedding_model)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [63]:
rag_instance=RAG(pdf_path= data_path, chunking_method='recursive', faiss_vector_store=faiss_vector_store)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [108]:
# rag_instance.search(query="total revenue of NAF 2023", method='ensemble')

In [65]:
def search(query : str):
    """
    This is a search function on a pre-exisiting financial knoweldge base, you can use this search function to search for financial information of specific companies you do not know, treat this function as a internal wiki search that you can use

    Args:
        query (str): input query            
    """
    results = rag_instance.search(query, method='ensemble')
    return json.dumps({"relevant information in the auditor notes": results})


In [109]:
# search('total revenue of NAF 2023')

In [67]:
search_tool = convert_to_tool(search)

In [71]:
search_tool.fn_signature

'{"name": "search", "description": "\\n    This is a search function on a pre-exisiting financial knoweldge base, you can use this search function to search for financial information of specific companies you do not know, treat this function as a internal wiki search that you can use\\n\\n    Args:\\n        query (str): input query            \\n    ", "parameters": {"properties": {"query": {"type": "str"}}}}'

In [88]:
weather_tool.fn_signature

'{"name": "get_current_weather", "description": "\\n    Get the current weather in a given location\\n    \\n    Args:\\n        location (str): The city and state, e.g. Madrid, Barcelona\\n        unit (str): The unit. It can take two values; \\"Felsius\\", \\"Fahrenheit\\"\\n    ", "parameters": {"properties": {"location": {"type": "str"}, "unit": {"type": "str"}}}}'

In [110]:
tool_agent = ToolAgent(tools=[weather_tool, search_tool])


In [111]:
temp=tool_agent.run("What is total revenue of National Ataxia Foundation in 2023")
print(temp)

[role: "user"
parts {
  text: "\nYou are a function calling and questions answering AI model. You are provided with function signatures within <tools></tools> XML tags.\nYou may call one or more functions to assist with the user query. Don\'t make assumptions about what values to plug into functions. Pay special attention to the properties \'types\'. You should use those types as in a Python dict.\n\nFor each function call return a json object with function name and arguments within <tool_call></tool_call>\nXML tags as follows:\n\n<tool_call>\n{\"name\": <function-name>,\"arguments\": <args-dict>,  \"id\": <monotonically-increasing-id>}\n</tool_call>\n\nHere are the available tools:\n\n<tools>\n{\"name\": \"get_current_weather\", \"description\": \"\\n    Get the current weather in a given location\\n    \\n    Args:\\n        location (str): The city and state, e.g. Madrid, Barcelona\\n        unit (str): The unit. It can take two values; \\\"Felsius\\\", \\\"Fahrenheit\\\"\\n    \", 

The total revenue of the National Ataxia Foundation in 2023 was $979,651.  This includes conference income, earned income, and investment income.  The total support and revenue, which includes net assets released from restrictions, was $4,184,787.



In [113]:
temp=tool_agent.run("What is the weather today in Charlotte")
print(temp)

[role: "user"
parts {
  text: "\nYou are a function calling and questions answering AI model. You are provided with function signatures within <tools></tools> XML tags.\nYou may call one or more functions to assist with the user query. Don\'t make assumptions about what values to plug into functions. Pay special attention to the properties \'types\'. You should use those types as in a Python dict.\n\nFor each function call return a json object with function name and arguments within <tool_call></tool_call>\nXML tags as follows:\n\n<tool_call>\n{\"name\": <function-name>,\"arguments\": <args-dict>,  \"id\": <monotonically-increasing-id>}\n</tool_call>\n\nHere are the available tools:\n\n<tools>\n{\"name\": \"get_current_weather\", \"description\": \"\\n    Get the current weather in a given location\\n    \\n    Args:\\n        location (str): The city and state, e.g. Madrid, Barcelona\\n        unit (str): The unit. It can take two values; \\\"Felsius\\\", \\\"Fahrenheit\\\"\\n    \", 

```tool_code
<tool_call>
{"name": "get_current_weather", "arguments": {"location": "Charlotte", "unit": "Fahrenheit"}, "id": 1}
</tool_call>
```The temperature in Charlotte is 25 Fahrenheit.


In [103]:
tool_agent = ToolAgent(tools=[weather_tool, search_tool])
temp=tool_agent.run("What is net assets of Wells Fargo in 2023")
print(temp)

[role: "user"
parts {
  text: "\nYou are a function calling and questions answering AI model. You are provided with function signatures within <tools></tools> XML tags.\nYou may call one or more functions to assist with the user query. Don\'t make assumptions about what values to plug into functions. Pay special attention to the properties \'types\'. You should use those types as in a Python dict.\n\nFor each function call return a json object with function name and arguments within <tool_call></tool_call>\nXML tags as follows:\n\n<tool_call>\n{\"name\": <function-name>,\"arguments\": <args-dict>,  \"id\": <monotonically-increasing-id>}\n</tool_call>\n\nHere are the available tools:\n\n<tools>\n{\"name\": \"get_current_weather\", \"description\": \"\\n    Get the current weather in a given location\\n    \\n    Args:\\n        location (str): The city and state, e.g. Madrid, Barcelona\\n        unit (str): The unit. It can take two values; \\\"Felsius\\\", \\\"Fahrenheit\\\"\\n    \", 

The provided text pertains to the National Ataxia Foundation, not Wells Fargo.  Therefore, I cannot answer your question about Wells Fargo's net assets using this information.  I need access to Wells Fargo's financial statements to provide that data.

